# Build model

In [1]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
from torch import nn
import time
from torch.utils.data.dataset import random_split

In [2]:
train_iter = AG_NEWS(split='train')

In [4]:
tokenizer = get_tokenizer('basic_english')
train_iter = AG_NEWS(split='train')
counter = Counter()
for (label, line) in train_iter:
    counter.update(tokenizer(line))
vocab = Vocab(counter, min_freq=1)

In [6]:
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: int(x) - 1

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter = AG_NEWS(split='train')
dataloader = DataLoader(train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch)

In [10]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [11]:
train_iter = AG_NEWS(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [12]:
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [13]:
from torch.utils.data.dataset import random_split
# Hyperparameters
EPOCHS = 2 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = AG_NEWS()
train_dataset = list(train_iter)
test_dataset = list(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.693
| epoch   1 |  1000/ 1782 batches | accuracy    0.853
| epoch   1 |  1500/ 1782 batches | accuracy    0.876
-----------------------------------------------------------
| end of epoch   1 | time: 10.60s | valid accuracy    0.884 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.898
| epoch   2 |  1000/ 1782 batches | accuracy    0.899
| epoch   2 |  1500/ 1782 batches | accuracy    0.904
-----------------------------------------------------------
| end of epoch   2 | time: 10.53s | valid accuracy    0.898 
-----------------------------------------------------------


In [14]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.891


# PREDICT FUNCTION

In [15]:
sm = torch.nn.Softmax()
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}
ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

In [16]:
def predict_fn(model, tokenizer, vocab, texts):
    with torch.no_grad():
        texts = [
            [vocab[token] for token in tokenizer(text)] 
            for text in texts]
        
        texts = torch.stack(
            [torch.tensor(text) for text in texts]
        )
        
        output = model(texts, None)
        return (
            sm(output).numpy().tolist(), 
            list(ag_news_label.values()),
            [0, 1, 2, 3]
        )

In [17]:
predict_fn(model, tokenizer, vocab, [ex_text_str, ex_text_str])

<ipython-input-16-07aa0e499264>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm(output).numpy().tolist(),


([[0.09169654548168182,
   0.8639219403266907,
   0.0048335157334804535,
   0.03954795002937317],
  [0.0916965901851654,
   0.8639219403266907,
   0.0048335157334804535,
   0.03954796493053436]],
 ['World', 'Sports', 'Business', 'Sci/Tec'],
 [0, 1, 2, 3])

# Unbox

### Pack model

In [18]:
import unboxapi
client = unboxapi.UnboxClient()

User is not logged in.


In [19]:
saved_path = client.pack_model(
    function=predict_fn, 
    model=model,
    tokenizer=tokenizer,
    vocab=vocab,
    model_name="TorchModel",
    model_type="pytorch"
)

[2021-05-12 13:25:40,419] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2021-05-12 13:25:41,976] INFO - BentoService bundle 'TorchModel:20210512132540_E7CBFD' saved to: /Users/gbayomi/bentoml/repository/TorchModel/20210512132540_E7CBFD


### Load

In [20]:
import bentoml
from pandas import DataFrame

In [21]:
bento_model = bentoml.load(saved_path)

[2021-05-12 13:25:59,636] WARNING - Module `template_model` already loaded, using existing imported module.
[2021-05-12 13:25:59,690] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
[2021-05-12 13:25:59,691] WARNING - pip package requirement torch already exist


### Test locally

In [22]:
bento_model.predict([{"text": "great"}])

<ipython-input-16-07aa0e499264>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  sm(output).numpy().tolist(),


[{'World': 3.307048075384955e-07,
  'Sports': 7.693777661188506e-06,
  'Business': 2.2019364678271813e-06,
  'Sci/Tec': 0.9999897480010986}]

In [23]:
bento_model.predictbatch([{"batch": ["great", "terrible"]}])

[[{'World': 3.307048075384955e-07,
   'Sports': 7.693770385230891e-06,
   'Business': 2.2019364678271813e-06,
   'Sci/Tec': 0.9999897480010986},
  {'World': 0.0010780768934637308,
   'Sports': 0.14372384548187256,
   'Business': 0.1999814510345459,
   'Sci/Tec': 0.655216634273529}]]

In [24]:
bento_model.predictactive([{"batch": ["great", "terrible"], "n_instances": 2}])

[([1, 0], ['terrible', 'great'])]

### Test as an endpoint

In [25]:
!bentoml run TorchModel:latest predict --input '{"text": "Which baking dish is best to bake a banana bread ?"}'

[2021-05-12 13:26:12,327] INFO - Getting latest version TorchModel:20210512132540_E7CBFD
[2021-05-12 13:26:13,693] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
<ipython-input-16-07aa0e499264>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
[2021-05-12 13:26:14,338] INFO - {'service_name': 'TorchModel', 'service_version': '20210512132540_E7CBFD', 'api': 'predict', 'task': {'data': '{"text": "Which baking dish is best to bake a banana bread ?"}', 'task_id': 'b4a9ff9c-cd1f-4ded-b446-d7979e51910f', 'cli_args': ('--input', '{"text": "Which baking dish is best to bake a banana bread ?"}'), 'inference_job_args': {}}, 'result':

In [26]:
!bentoml run TorchModel:latest predictbatch --input '{"batch": ["great", "terrible"]}'

[2021-05-12 13:26:18,773] INFO - Getting latest version TorchModel:20210512132540_E7CBFD
[2021-05-12 13:26:20,039] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
<ipython-input-16-07aa0e499264>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
[2021-05-12 13:26:20,553] INFO - {'service_name': 'TorchModel', 'service_version': '20210512132540_E7CBFD', 'api': 'predictbatch', 'task': {'data': '{"batch": ["great", "terrible"]}', 'task_id': 'ac5b0c82-d87d-4409-bb5b-c49928dc06e0', 'cli_args': ('--input', '{"batch": ["great", "terrible"]}'), 'inference_job_args': {}}, 'result': {'data': '[{"World": 3.307048075384955e-07, "Sports": 

In [27]:
!bentoml run TorchModel:latest predictactive --input '{"batch": ["great", "terrible"], "n_instances": 2}'

[2021-05-12 13:26:23,629] INFO - Getting latest version TorchModel:20210512132540_E7CBFD
[2021-05-12 13:26:24,873] WARNING - BentoML by default does not include spacy and torchvision package when using PytorchModelArtifact. To make sure BentoML bundle those packages if they are required for your model, either import those packages in BentoService definition file or manually add them via `@env(pip_packages=['torchvision'])` when defining a BentoService
<ipython-input-16-07aa0e499264>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
[2021-05-12 13:26:25,387] INFO - {'service_name': 'TorchModel', 'service_version': '20210512132540_E7CBFD', 'api': 'predictactive', 'task': {'data': '{"batch": ["great", "terrible"], "n_instances": 2}', 'task_id': '44cb616a-a63a-45e7-ae21-3f02553711ca', 'cli_args': ('--input', '{"batch": ["great", "terrible"], "n_instances": 2}'), 'inference_job_args': {}}, 'result': {'data': '[[1, 0]